In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Math

# Physical constants
e0 = 8.854e-12  # permittivity of free space (F/m)
mu0 = 4 * np.pi * 1e-7  # permeability of free space (H/m)

# Transmission line model (R, L, C, G per unit length)
def get_C_circ(e0, e_ins, D, d_ins):
    return 2 * np.pi * e0 * e_ins / np.log((D/2 + d_ins) / (D/2))

def get_L_circ(mu_ins, D, d_ins):
    return mu_ins / (2 * np.pi) * np.log((D/2 + d_ins) / (D/2))

def get_G_circ(sig_ins, e_ins, C):
    return sig_ins / e_ins * C

def calc_R_tot(freq, D, sig_cu, mu_cu, sig_epoxy, mu_epoxy, d_epoxy, d_ins):
    delta_cu = (sig_cu * mu_cu * freq * np.pi) ** -0.5
    delta_1 = np.minimum(delta_cu, D/2)
    delta_epoxy = (sig_epoxy * mu_epoxy * freq * np.pi) ** -0.5
    delta_2 = np.minimum(delta_epoxy, d_epoxy)
    R_IC = 1/sig_cu / ((D/2)**2 - (D/2 - delta_1)**2)
    R_OC = 1/sig_epoxy / (((D/2 + d_ins + delta_2)**2) - ((D/2 + d_ins)**2))
    return R_IC + R_OC

def attenuation(freq, z, D, d_ins, d_epoxy, e_ins, sig_ins, sig_cu, sig_epoxy,
                mu_ins=mu0, mu_cu=mu0, mu_epoxy=mu0):
    R = calc_R_tot(freq, D, sig_cu, mu_cu, sig_epoxy, mu_epoxy, d_epoxy, d_ins)
    C = get_C_circ(e0, e_ins, D, d_ins)
    L = get_L_circ(mu_ins, D, d_ins)
    G = get_G_circ(sig_ins, e_ins, C)
    gamma = np.sqrt((R + 2j*np.pi*freq*L) * (G + 2j*np.pi*freq*C))
    alpha = np.real(gamma)
    att = -8.686 * alpha * z   # negative attenuation (loss)
    return att

# Multi-plot comparison system
comparisons = []
_next_id = 1

def _refresh_dropdown():
    if comparisons:
        options = [(f"{c['label']} (Length={c['length']} m)", c['id']) for c in comparisons]
        remove_dropdown.options = options
        if remove_dropdown.value not in [c['id'] for c in comparisons]:
            remove_dropdown.value = comparisons[0]['id']
        remove_dropdown.disabled = False
        remove_button.disabled = False
    else:
        remove_dropdown.options = [("— no saved curves —", None)]
        remove_dropdown.value = None
        remove_dropdown.disabled = True
        remove_button.disabled = True

def plot_side_by_side(length, D_mm, d_ins_um, d_epoxy_mm,
                      e_ins, sig_ins, sig_cu, sig_epoxy,
                      ymin=None, ymax=None):
    # Convert geometry
    D = D_mm * 1e-3
    d_ins = d_ins_um * 1e-6
    d_epoxy = d_epoxy_mm * 1e-3
    r_wire = D / 2
    r_ins = r_wire + d_ins
    r_epoxy = r_ins + d_epoxy
    
    # Frequency sweep
    f = np.logspace(4, 11, 1000)  # 10 kHz to 100 GHz
    att = attenuation(f, length, D, d_ins, d_epoxy,
                      e_ins, sig_ins, sig_cu, sig_epoxy)
    
    # Make side-by-side plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,6))
    
    # Geometry schematic
    ax1.set_aspect("equal")
    wire = plt.Circle((0,0), r_wire, color="orange", label="Copper wire")
    ins  = plt.Circle((0,0), r_ins, fill=False, color="blue", linestyle="--", label="Insulation")
    epoxy= plt.Circle((0,0), r_epoxy, fill=False, color="grey", linestyle="-.", label="Epoxy")
    ax1.add_patch(wire)
    ax1.add_patch(ins)
    ax1.add_patch(epoxy)
    ax1.set_xlim(-r_epoxy*1.2, r_epoxy*1.2)
    ax1.set_ylim(-r_epoxy*1.2, r_epoxy*1.2)
    ax1.set_xlabel("x (m)")
    ax1.set_ylabel("y (m)")
    ax1.set_title("Cross-section")
    ax1.legend()
    
    # Attenuation curve
    for c in comparisons:
        f_c, att_c = plot_curve(c['length'], c['D_mm'], c['d_ins_um'], c['d_epoxy_mm'],
                                c['e_ins'], c['sig_ins'], c['sig_cu'], c['sig_epoxy'],
                                plot_only=True)
        ax2.semilogx(f_c, att_c, lw=2, label=c['label'])
    ax2.semilogx(f, att, "--", color="black", label=f"Current (Length={length:.2f} m)")
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Attenuation (dB)")
    ax2.set_title("Full RLCG Filter Attenuation")
    ax2.grid(True, which="both", ls="--")
    ax2.invert_yaxis()  # 0 at top, more negative below
    if ymin is not None and ymax is not None:
        ax2.set_ylim(ymin, ymax)
    else:
        ax2.set_ylim(0, -200)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

def plot_curve(length, D_mm, d_ins_um, d_epoxy_mm,
               e_ins, sig_ins, sig_cu, sig_epoxy,
               ymin=None, ymax=None, plot_only=False):
    D = D_mm * 1e-3
    d_ins = d_ins_um * 1e-6
    d_epoxy = d_epoxy_mm * 1e-3
    f = np.logspace(4, 11, 1000)  # 10 kHz to 100 GHz
    att = attenuation(f, length, D, d_ins, d_epoxy,
                      e_ins, sig_ins, sig_cu, sig_epoxy)
    if plot_only:
        return f, att

def add_comparison(length, D_mm, d_ins_um, d_epoxy_mm,
                   e_ins, sig_ins, sig_cu, sig_epoxy, label):
    global _next_id
    comparisons.append({
        "id": _next_id, "label": label or f"Filter {_next_id}",
        "length": length, "D_mm": D_mm, "d_ins_um": d_ins_um, "d_epoxy_mm": d_epoxy_mm,
        "e_ins": e_ins, "sig_ins": sig_ins, "sig_cu": sig_cu, "sig_epoxy": sig_epoxy,
    })
    _next_id += 1
    _refresh_dropdown()
    _redraw()

def remove_selected(_):
    sel_id = remove_dropdown.value
    if sel_id is None: return
    idx = next((i for i, c in enumerate(comparisons) if c['id'] == sel_id), None)
    if idx is not None: comparisons.pop(idx)
    _refresh_dropdown()
    _redraw()

def reset_comparisons(_):
    comparisons.clear()
    _refresh_dropdown()
    _redraw()

# Widgets
common_style = {'description_width': 'initial'}
common_layout = widgets.Layout(width="500px")

# Parameter sliders (defaults as before)
wire_length_slider = widgets.FloatSlider(value=1.5, min=0.1, max=5.0, step=0.01,
    description="Wire Length (m)", style=common_style, layout=common_layout)
wire_diameter_slider = widgets.FloatSlider(value=0.097, min=0.01, max=1.0, step=0.001,
    description="Wire Diameter (mm)", readout_format=".3f", style=common_style, layout=common_layout)
insulation_thickness_slider = widgets.FloatSlider(value=8.0, min=0.1, max=50.0, step=0.1,
    description="Insulation Thickness (µm)", readout_format=".1f", style=common_style, layout=common_layout)
epoxy_thickness_slider = widgets.FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1,
    description="Epoxy Thickness (mm)", readout_format=".2f", style=common_style, layout=common_layout)
dielectric_constant_slider = widgets.FloatSlider(value=4.6, min=1.0, max=10.0, step=0.1,
    description="Dielectric Constant of Insulation (er)", style=common_style, layout=common_layout)
conductivity_insulation_slider = widgets.FloatLogSlider(value=6.3e-13, base=10, min=-15, max=-10, step=0.01,
    description="Conductivity of Insulation (S/m)", readout_format=".1e", style=common_style, layout=common_layout)
conductivity_copper_slider = widgets.FloatSlider(value=5.95e7, min=5.0e7, max=6.5e7, step=1e5,
    description="Conductivity of Copper (S/m)", readout_format=".2e", style=common_style, layout=common_layout)
conductivity_epoxy_slider = widgets.FloatLogSlider(value=2.0e5, base=10, min=4, max=7, step=0.01,
    description="Conductivity of Epoxy (S/m)", readout_format=".1e", style=common_style, layout=common_layout)

# Y-axis sliders
ymin_slider = widgets.FloatSlider(value=-200, min=-5000, max=0, step=1,
    description="Y-axis Min (dB)", style=common_style, layout=common_layout)
ymax_slider = widgets.FloatSlider(value=0, min=-5000, max=0, step=1,
    description="Y-axis Max (dB)", style=common_style, layout=common_layout)

label_text = widgets.Text(value="", placeholder="Enter filter name", description="Label:",
    style=common_style, layout=common_layout)

add_button = widgets.Button(description="Add Curve", button_style="success")
reset_button = widgets.Button(description="Reset", button_style="warning")
remove_dropdown = widgets.Dropdown(description="Remove:", layout=widgets.Layout(width="300px"))
remove_button = widgets.Button(description="Remove Selected", button_style="danger")
refresh_button = widgets.Button(description="Refresh", button_style="info")

# Actions
add_button.on_click(lambda _: add_comparison(
    wire_length_slider.value, wire_diameter_slider.value,
    insulation_thickness_slider.value, epoxy_thickness_slider.value,
    dielectric_constant_slider.value,
    conductivity_insulation_slider.value,
    conductivity_copper_slider.value,
    conductivity_epoxy_slider.value,
    label_text.value))
reset_button.on_click(reset_comparisons)
remove_button.on_click(remove_selected)
refresh_button.on_click(lambda _: _redraw())

_refresh_dropdown()

out = widgets.Output()

def _redraw(*args):
    with out:
        out.clear_output(wait=True)
        # equation
        display(Math(r"\text{Attenuation: } \; A(f) = -8.686 \cdot \alpha(f) \cdot z, \quad \alpha = \Re\{\gamma\}, \; \gamma = \sqrt{(R+j\omega L)(G+j\omega C)}"))
        # side by side plots
        plot_side_by_side(wire_length_slider.value, wire_diameter_slider.value,
                          insulation_thickness_slider.value, epoxy_thickness_slider.value,
                          dielectric_constant_slider.value,
                          conductivity_insulation_slider.value,
                          conductivity_copper_slider.value,
                          conductivity_epoxy_slider.value,
                          ymin=ymin_slider.value, ymax=ymax_slider.value)

for w in (wire_length_slider, wire_diameter_slider,
          insulation_thickness_slider, epoxy_thickness_slider,
          dielectric_constant_slider,
          conductivity_insulation_slider, conductivity_copper_slider,
          conductivity_epoxy_slider,
          ymin_slider, ymax_slider):
    w.observe(_redraw, names="value")

_redraw()

controls = widgets.VBox([
    wire_length_slider,
    wire_diameter_slider,
    insulation_thickness_slider,
    epoxy_thickness_slider,
    dielectric_constant_slider,
    conductivity_insulation_slider,
    conductivity_copper_slider,
    conductivity_epoxy_slider,
    ymin_slider, ymax_slider,
    label_text,
    widgets.HBox([add_button, reset_button, refresh_button]),
    widgets.HBox([remove_dropdown, remove_button]),
])

display(controls, out)


Output()